In [1]:
import os

In [2]:
! pwd

/home/gautham/E2EProjects/TextSummarisation/research


In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
type(CONFIG_FILE_PATH)

pathlib.PosixPath

In [7]:
class ConfigarationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAMS_FILE_FILE):
        

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(param_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """ 
        Returns : DataIngestionConfig
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Download the file from the source URL and save it locally under the 
        specified root directory.
        """
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(url = self.config.source_URL,
                                       filename = self.config.local_data_file)
            logger.info(f"{filename} Download with following info : \n{header} \nDownloaded {get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"{self.config.local_data_file} already exists. \nSize : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extract the zip file to the specified root directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [24]:
try:
    config = ConfigarationManager()
    data_ingestion = DataIngestion(config.get_data_ingestion_config())
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-10 09:06:32,779: INFO: common: yaml file : config/config.yaml loded successfully]
[2024-03-10 09:06:32,785: INFO: common: yaml file : params.yaml loded successfully]
[2024-03-10 09:06:32,788: INFO: common: Created directory at : artifacts]
[2024-03-10 09:06:32,790: INFO: common: Created directory at : artifacts/data_ingesition]
[2024-03-10 09:06:43,613: INFO: 3014375468: artifacts/data_ingesition/data.zip Download with following info : 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3F50:3F1C4F:534436:5D1DD1:65ED29B8
Accept-Ranges: bytes
Date: Sun, 10 Mar 2024 03:36:33 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4730-BOM
X-Cache